In [1]:
import os
import json
import asyncio
import datetime

from dotenv import load_dotenv, find_dotenv

from IPython.display import Image, display

from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.models import (
    FileInfo, FileSearchTool, VectorStore,
    CodeInterpreterTool, FilePurpose,
    ListSortOrder
)

from semantic_kernel.agents import (
    ChatCompletionAgent, ChatHistoryAgentThread,
    AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, AuthorRole, TextContent
)


In [2]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

In [3]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")

In [4]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Azure Chat Completion の初期化
azure_chat_completion = AzureChatCompletion(
    api_key=AZURE_OPENAI_API_KEY,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
)

In [5]:
class OrderStatusPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        return f"Order {order_id} is shipped and will arrive in 2-3 days."

class OrderRefundPlugin:
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        print(f"Processing refund for order {order_id} due to: {reason}")
        return f"Refund for order {order_id} has been processed successfully."

class OrderReturnPlugin:
    @kernel_function
    def process_return(self, order_id: str, reason: str) -> str:
        print(f"Processing return for order {order_id} due to: {reason}")
        return f"Return for order {order_id} has been processed successfully."

# エージェントの定義
support_agent = ChatCompletionAgent(
    name="TriageAgent",
    description="問い合わせ内容を振り分けるカスタマーサポート担当者です。",
    instructions="お客様からのご要望やお問い合わせに対応してください。",
    service=azure_chat_completion,
)

refund_agent = ChatCompletionAgent(
    name="RefundAgent",
    description="返金対応を担当するカスタマーサポート担当者です。",
    instructions="返金に関するご要望に対応してください。",
    service=azure_chat_completion,
    plugins=[OrderRefundPlugin()],
)

order_status_agent = ChatCompletionAgent(
    name="OrderStatusAgent",
    description="注文状況の確認を担当するカスタマーサポート担当者です。",
    instructions="注文状況の確認に関するご要望に対応してください。",
    service=azure_chat_completion,
    plugins=[OrderStatusPlugin()],
)

order_return_agent = ChatCompletionAgent(
    name="OrderReturnAgent",
    description="返品対応を担当するカスタマーサポート担当者です。",
    instructions="返品に関するご要望に対応してください。",
    service=azure_chat_completion,
    plugins=[OrderReturnPlugin()],
)


handoffs = (
    OrchestrationHandoffs()
    .add_many(
        source_agent=support_agent.name,
        target_agents={
            refund_agent.name: "「返金関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_status_agent.name: " 「注文ステータス」の問題が発生した場合は、このエージェントにハンズオフしてください",
            order_return_agent.name: "「返品関連」の問題が発生した場合は、このエージェントにハンズオフしてください",
        },
    )
    .add(
        source_agent=refund_agent.name,
        target_agent=support_agent.name,
        description="「返金関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_status_agent.name,
        target_agent=support_agent.name,
        description="「注文ステータス」の問題ではない場合、このエージェントにハンズオフしてください",
    )
    .add(
        source_agent=order_return_agent.name,
        target_agent=support_agent.name,
        description="「返品関連」の問題ではない場合、このエージェントにハンズオフしてください",
    )
)

In [6]:
def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"{message.name}: {message.content}")

def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    print(f"User: {user_input}")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)



In [7]:
async def run_handoff():
    # エージェントの定義
    handoff_orchestration = HandoffOrchestration(
        members=[
            support_agent, 
            refund_agent, 
            order_status_agent, 
            order_return_agent
        ],
        handoffs=handoffs,
        agent_response_callback=agent_response_callback,
        human_response_function=human_response_function,
    )
    runtime = InProcessRuntime()
    runtime.start()

    orchestration_result = await handoff_orchestration.invoke(
        task="お客様から電話です。",
        runtime=runtime,
    )

    value = await orchestration_result.get()
    print(value)

    await runtime.stop_when_idle()



In [9]:
await run_handoff()

TriageAgent: ご連絡いただきありがとうございます。お客様からの電話について、どのようなご用件でしょうか？内容（例：注文に関する質問、返品・返金のご希望など）をお知らせいただけますか？ご案内できるようサポートいたします。
User: 発送状況について
TriageAgent: 
TriageAgent: 
TriageAgent: 
OrderStatusAgent: かしこまりました。  
ご注文の発送状況を確認いたします。

お手数ですが、お客様の「注文番号」を教えていただけますか？  
注文番号がわかれば、すぐに発送状況をお調べいたします。
User: 123
OrderStatusAgent: 
OrderStatusAgent: 
OrderStatusAgent: ご注文番号「123」は、すでに発送済みです。  
商品は2～3日以内にお届け予定ですので、到着まで今しばらくお待ちください。

他にご不明な点やご質問がございましたらお知らせください。
User: ありがとうございます。345を返品したいです。
OrderStatusAgent: 
OrderStatusAgent: 
OrderStatusAgent: 
TriageAgent: 
TriageAgent: 
TriageAgent: 
OrderReturnAgent: 返品のご希望を承りました。ご注文番号「345」の返品手続きを進めます。

恐れ入りますが、返品理由をお伺いしてもよろしいでしょうか？  
（例：商品不良、注文間違い、イメージと違った等）

理由を教えていただければ、速やかに対応いたします。
User: 商品不良
Processing return for order 345 due to: 商品不良
OrderReturnAgent: 
OrderReturnAgent: 
OrderReturnAgent: ご注文番号「345」の返品を「商品不良」との理由で手続きいたしました。

返品手続きは完了しております。今後の流れなどご不明な点がありましたらお気軽にお知らせください。

本日はご連絡ありがとうございました。
User: 他に何ができますか？
OrderReturnAgent: お問い合わせありがとうございます。私（オペレーター）では、主に下記のようなご対